# 01 - Data Exploration: TCGA Pan-Cancer Gene Expression

## Overview
In this notebook, we'll explore The Cancer Genome Atlas (TCGA) Pan-Cancer gene expression dataset. This is real data from tumor samples, where each sample has expression levels for ~20,000 genes.

**Goal**: Understand the data structure and visualize patterns that distinguish cancer types.

### What is Gene Expression?
- Genes are segments of DNA that code for proteins
- Gene expression measures how "active" each gene is (how much RNA is being produced)
- Different cancers have different expression patterns - this is what we'll learn to classify!

### Dataset Details
- **Source**: TCGA via UCI ML Repository
- **Samples**: 801 tumor samples
- **Features**: 20,531 genes
- **Classes**: 5 cancer types (BRCA, KIRC, LUAD, PRAD, COAD)

## 1. Setup and Imports

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For dimensionality reduction visualization
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Add parent directory to path to import our modules
import sys
sys.path.append('..')
from src.data_loader import load_tcga_data, load_as_dataframe, CANCER_TYPE_INFO

# Set style for plots
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

print("Libraries loaded successfully!")

## 2. Load the Data

Let's download and load the TCGA dataset. This may take a minute on first run.

In [ ]:
# Load the data
X, y, gene_names, sample_ids = load_tcga_data(verbose=True)

print(f"\n" + "="*50)
print("DATA SUMMARY")
print("="*50)
print(f"Number of samples: {X.shape[0]}")
print(f"Number of genes (features): {X.shape[1]}")
print(f"Number of cancer types: {len(np.unique(y))}")

In [ ]:
# Also load as DataFrame for easier exploration
df_expression, df_labels = load_as_dataframe()

# Combine into single dataframe for analysis
df = df_expression.copy()
df['cancer_type'] = df_labels['Class'].values

print("DataFrame shape:", df.shape)
df.head()

## 3. Understanding the Cancer Types

Let's look at what cancers we're classifying:

In [ ]:
# Display cancer type information
print("CANCER TYPES IN DATASET")
print("=" * 60)
for code, info in CANCER_TYPE_INFO.items():
    print(f"\n{code}: {info['full_name']}")
    print(f"   Organ: {info['organ']}")
    print(f"   Note: {info['description']}")

In [ ]:
# Class distribution
class_counts = pd.Series(y).value_counts().sort_index()

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(class_counts.index, class_counts.values, color=sns.color_palette('husl', 5))

# Add value labels on bars
for bar, count in zip(bars, class_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            str(count), ha='center', va='bottom', fontsize=12)

ax.set_xlabel('Cancer Type', fontsize=12)
ax.set_ylabel('Number of Samples', fontsize=12)
ax.set_title('Class Distribution in TCGA Dataset', fontsize=14)

# Add full names as x-tick labels
ax.set_xticklabels([f"{code}\n({CANCER_TYPE_INFO[code]['organ']})" for code in class_counts.index])

plt.tight_layout()
plt.savefig('../results/figures/class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTotal samples: {class_counts.sum()}")
print(f"Class balance: {class_counts.min()}/{class_counts.max()} = {class_counts.min()/class_counts.max():.2f}")

## 4. Exploring Gene Expression Values

Let's understand the distribution of expression values:

In [ ]:
# Basic statistics of expression values
print("GENE EXPRESSION STATISTICS")
print("=" * 40)
print(f"Min value: {X.min():.2f}")
print(f"Max value: {X.max():.2f}")
print(f"Mean value: {X.mean():.2f}")
print(f"Median value: {np.median(X):.2f}")
print(f"Std deviation: {X.std():.2f}")
print(f"\nZero values: {(X == 0).sum()} ({(X == 0).sum()/(X.size)*100:.1f}%)")

In [ ]:
# Distribution of expression values
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall distribution
ax1 = axes[0]
ax1.hist(X.flatten(), bins=100, alpha=0.7, color='steelblue', edgecolor='black', linewidth=0.5)
ax1.set_xlabel('Expression Value', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Distribution of All Gene Expression Values', fontsize=14)
ax1.axvline(X.mean(), color='red', linestyle='--', label=f'Mean: {X.mean():.1f}')
ax1.legend()

# Log-transformed (common in genomics)
ax2 = axes[1]
X_log = np.log2(X + 1)  # Add 1 to handle zeros
ax2.hist(X_log.flatten(), bins=100, alpha=0.7, color='forestgreen', edgecolor='black', linewidth=0.5)
ax2.set_xlabel('Log2(Expression + 1)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Log-Transformed Expression Values', fontsize=14)

plt.tight_layout()
plt.savefig('../results/figures/expression_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Gene Expression Variability

Not all genes are equally informative. Let's look at which genes have high variability across samples (these are likely more useful for classification):

In [ ]:
# Calculate variance for each gene
gene_variance = np.var(X, axis=0)
gene_mean = np.mean(X, axis=0)

# Create a dataframe for analysis
df_genes = pd.DataFrame({
    'gene': gene_names,
    'variance': gene_variance,
    'mean': gene_mean,
    'cv': np.sqrt(gene_variance) / (gene_mean + 1e-10)  # Coefficient of variation
})

print("TOP 20 MOST VARIABLE GENES")
print("=" * 50)
print("(These genes show the most variation across tumor samples)")
print()
print(df_genes.nlargest(20, 'variance')[['gene', 'variance', 'mean']].to_string(index=False))

In [ ]:
# Visualize variance distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Variance distribution
ax1 = axes[0]
ax1.hist(np.log10(gene_variance + 1), bins=50, alpha=0.7, color='purple', edgecolor='black')
ax1.set_xlabel('Log10(Variance + 1)', fontsize=12)
ax1.set_ylabel('Number of Genes', fontsize=12)
ax1.set_title('Distribution of Gene Variance', fontsize=14)

# Mean vs Variance (common quality check in RNA-seq)
ax2 = axes[1]
ax2.scatter(np.log10(gene_mean + 1), np.log10(gene_variance + 1), alpha=0.1, s=1)
ax2.set_xlabel('Log10(Mean Expression + 1)', fontsize=12)
ax2.set_ylabel('Log10(Variance + 1)', fontsize=12)
ax2.set_title('Mean-Variance Relationship', fontsize=14)

plt.tight_layout()
plt.savefig('../results/figures/gene_variance.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nGenes with zero variance (constant): {(gene_variance == 0).sum()}")
print(f"Genes with very low variance (<0.1): {(gene_variance < 0.1).sum()}")

## 6. Dimensionality Reduction: PCA

With 20,000+ genes, we can't visualize the data directly. PCA (Principal Component Analysis) reduces dimensions while preserving the most important patterns.

**If cancer types form distinct clusters in PCA space, that's a good sign our ML models will work well!**

In [ ]:
# Standardize the data (important for PCA)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=50)  # Keep top 50 components
X_pca = pca.fit_transform(X_scaled)

print(f"Original shape: {X.shape}")
print(f"PCA shape: {X_pca.shape}")
print(f"\nVariance explained by first 10 components:")
for i in range(10):
    print(f"  PC{i+1}: {pca.explained_variance_ratio_[i]*100:.1f}%")
print(f"\nTotal variance explained (top 50): {pca.explained_variance_ratio_.sum()*100:.1f}%")

In [ ]:
# Visualize PCA results
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Color map for cancer types
cancer_types = np.unique(y)
colors = sns.color_palette('husl', len(cancer_types))
color_map = dict(zip(cancer_types, colors))

# PC1 vs PC2
ax1 = axes[0]
for cancer in cancer_types:
    mask = y == cancer
    ax1.scatter(X_pca[mask, 0], X_pca[mask, 1], 
                c=[color_map[cancer]], label=cancer, alpha=0.6, s=50)
ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
ax1.set_title('PCA: Cancer Types in Gene Expression Space', fontsize=14)
ax1.legend(title='Cancer Type')

# Scree plot
ax2 = axes[1]
cumulative_var = np.cumsum(pca.explained_variance_ratio_) * 100
ax2.bar(range(1, 21), pca.explained_variance_ratio_[:20] * 100, alpha=0.7, label='Individual')
ax2.plot(range(1, 21), cumulative_var[:20], 'ro-', label='Cumulative')
ax2.axhline(y=90, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Principal Component', fontsize=12)
ax2.set_ylabel('Variance Explained (%)', fontsize=12)
ax2.set_title('Scree Plot: Variance Explained by PCs', fontsize=14)
ax2.legend()

plt.tight_layout()
plt.savefig('../results/figures/pca_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

# Find how many components needed for 90% variance
n_components_90 = np.argmax(cumulative_var >= 90) + 1
print(f"\nComponents needed for 90% variance: {n_components_90}")

## 7. Expression Patterns by Cancer Type

Let's look at how expression differs between cancer types for the most variable genes:

In [ ]:
# Get top 20 most variable genes
top_var_genes = df_genes.nlargest(20, 'variance')['gene'].values
top_var_idx = [gene_names.index(g) for g in top_var_genes]

# Create heatmap data
df_heatmap = pd.DataFrame(X[:, top_var_idx], columns=top_var_genes)
df_heatmap['cancer_type'] = y

# Calculate mean expression per cancer type
mean_expression = df_heatmap.groupby('cancer_type').mean()

# Plot heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(mean_expression.T, cmap='RdBu_r', center=0, 
            annot=False, fmt='.1f', cbar_kws={'label': 'Mean Expression'})
plt.title('Mean Expression of Top Variable Genes by Cancer Type', fontsize=14)
plt.xlabel('Cancer Type', fontsize=12)
plt.ylabel('Gene', fontsize=12)
plt.tight_layout()
plt.savefig('../results/figures/gene_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Box plots for a few interesting genes
interesting_genes = top_var_genes[:6]  # Top 6 most variable

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, gene in enumerate(interesting_genes):
    gene_idx = gene_names.index(gene)
    data = pd.DataFrame({'Expression': X[:, gene_idx], 'Cancer Type': y})
    
    sns.boxplot(data=data, x='Cancer Type', y='Expression', ax=axes[i], palette='husl')
    axes[i].set_title(f'{gene}', fontsize=12)
    axes[i].set_xlabel('')
    
plt.suptitle('Expression Distribution of Top Variable Genes', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('../results/figures/gene_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Sample Correlations

Let's see if samples of the same cancer type are more similar to each other:

In [ ]:
# Subsample for visualization (correlation matrix of 801x801 is hard to see)
np.random.seed(42)
sample_idx = []
for cancer in cancer_types:
    cancer_idx = np.where(y == cancer)[0]
    sample_idx.extend(np.random.choice(cancer_idx, min(30, len(cancer_idx)), replace=False))

X_sample = X_scaled[sample_idx]
y_sample = y[sample_idx]

# Compute correlation matrix
corr_matrix = np.corrcoef(X_sample)

# Sort by cancer type for better visualization
sort_idx = np.argsort(y_sample)
corr_matrix_sorted = corr_matrix[sort_idx][:, sort_idx]
y_sorted = y_sample[sort_idx]

# Plot
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix_sorted, cmap='coolwarm', center=0, 
            xticklabels=False, yticklabels=False)

# Add cancer type labels
unique_sorted = np.unique(y_sorted)
boundaries = [0]
for cancer in unique_sorted:
    boundaries.append(boundaries[-1] + (y_sorted == cancer).sum())

for i, cancer in enumerate(unique_sorted):
    mid = (boundaries[i] + boundaries[i+1]) / 2
    plt.text(-5, mid, cancer, ha='right', va='center', fontsize=10)
    plt.text(mid, -5, cancer, ha='center', va='bottom', fontsize=10, rotation=90)

plt.title('Sample Correlation Matrix (grouped by cancer type)', fontsize=14)
plt.tight_layout()
plt.savefig('../results/figures/sample_correlation.png', dpi=150, bbox_inches='tight')
plt.show()

print("Notice how samples of the same cancer type (blocks along diagonal)")
print("tend to be more correlated with each other!")

## 9. Key Observations Summary

Let's summarize what we learned about the data:

In [ ]:
print("="*60)
print("DATA EXPLORATION SUMMARY")
print("="*60)
print()
print("DATASET CHARACTERISTICS:")
print(f"  - {X.shape[0]} tumor samples from 5 cancer types")
print(f"  - {X.shape[1]:,} gene expression features")
print(f"  - Reasonably balanced classes (ratio: {class_counts.min()/class_counts.max():.2f})")
print()
print("KEY FINDINGS:")
print("  1. Expression values are right-skewed (many zeros/low values)")
print("     -> Log transformation will help normalization")
print()
print("  2. High dimensionality (20K+ features, only 801 samples)")
print("     -> Feature selection/regularization will be important")
print(f"     -> {n_components_90} PCA components capture 90% variance")
print()
print("  3. Cancer types show distinct clusters in PCA space")
print("     -> Good sign for classification!")
print()
print("  4. Many genes have near-zero variance (not informative)")
print(f"     -> {(gene_variance < 0.1).sum()} genes have variance < 0.1")
print()
print("  5. Samples of same cancer type are more correlated")
print("     -> Biological signal is present in the data")
print()
print("NEXT STEPS (Notebook 02):")
print("  - Data preprocessing and normalization")
print("  - Feature selection to reduce dimensionality")
print("  - Train/test splitting with stratification")

In [ ]:
# Save some processed data for next notebook
np.save('../data/processed/X_scaled.npy', X_scaled)
np.save('../data/processed/X_pca.npy', X_pca)
np.save('../data/processed/y.npy', y)
np.save('../data/processed/gene_names.npy', gene_names)

print("Processed data saved to data/processed/")